# PART 2 - Text and Metadata extraction from the audio of the video file

In the following section of the workshop, we're going to:
- Transcribe the file's audio into text using Amazon Transcribe
- Prepare the transcript data in the format expected by Amazon Comprehend
- Run a topic modelling job using Amazon Comprehend to extract topics
- Run an NER (Named Entity Recognition) job using Amazon Comprehend to extract names and entities (e.g. countries, places, etc)
<br>

All those metadata will then be used alongside with metadata extracted via computer vision with Rekognition to populate our knowlege graph in part 3.
<br>

In [ ]:
#load stored variable from previous notebooks
%store -r

In [ ]:
#installing a useful library to manipulate json object
!pip install jsonlines
!pip install pandas
!pip install boto3

## Transcribe the file's audio into text
Amazon Transcribe uses machine learning to recognize speech in audio and video files and transcribe that speech into text. Practical use cases for Amazon Transcribe include transcriptions of customer-agent calls and closed captions for videos.

https://docs.aws.amazon.com/transcribe/latest/dg/transcribe-whatis.html

In [ ]:
import boto3
import os
import random
import time
import urllib
import json
import csv
import tarfile
import pandas as pd
import jsonlines

In [ ]:
transcribe = boto3.client('transcribe')

#creating a unique name for the job
transcribe_job_name = "transcribe_job_knowledge_graph" + str(random.randint(0, 100000))

#s3 path to your video file
transcribe_job_uri = "s3://" + os.path.join(bucket, s3_video_input_path, video_file)

#starting the transcription job
transcription_job = transcribe.start_transcription_job(
    TranscriptionJobName=transcribe_job_name,
    Media={'MediaFileUri': transcribe_job_uri},
    MediaFormat='mp4',
    LanguageCode='en-US',
    OutputBucketName=bucket
)

Monitoring the job's completion

In [ ]:
print(transcribe_job_name)
while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=transcribe_job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print(".", end='')
    time.sleep(5)
print(status['TranscriptionJob']['TranscriptionJobStatus'])

Download the transcript file from the s3 bucket

In [ ]:
s3 = boto3.Session().resource('s3')

#retrieving the transcript file URI
s3_transcript_file_url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']

S3_transcript_file_name = s3_transcript_file_url.split('/')[-1]

#local path where to store the transcript file
local_transcribe_file_path = os.path.join(tmp_local_folder, S3_transcript_file_name)

#downloading locally the transcript file
s3.Bucket(bucket).Object(S3_transcript_file_name).download_file(local_transcribe_file_path)

In [ ]:
#loading the transcript object in memory
transcribe_file = open(local_transcribe_file_path)
transcribe_json_data = json.load(transcribe_file)

Let's have a look at the output. below is the itemised version of the transcript, word by word for the 5 first words.

In [ ]:
transcript_items = transcribe_json_data['results']['items']
transcript_items[:5]

## Formating the transcript to be consumed by Amazon Comprehend for the following 2 jobs.
The documentation explains that we can format the input CSV file in 2 ways. Either we provide one document per file or a file containing one document per line. We're going to pick the latter option.

We have different ways of splitting that text into "blocks" of words. One logical way of doing it could be to do it sentence by sentence.<br>
We're choosing here to segment our text transcript by chunk of 1 minute.<br>
Reason being that later we're going to attach video/audio metadata to 1 minute video segments in order to have a fine grained level of information on our video. <br>

In [ ]:
#setting the size of our segments to 1 minute
segment_size_ms = 60000

The following function is using the timestamp from each item to break the whole transcript into 1min chunks.

In [ ]:
def prepare_transcribed_text_for_topic_modelling(transcript_items, segment_size_ms=60000):

    #initiatlising current segment with segment size
    current_segment_end = segment_size_ms
    sentence_list_per_segment = []
    buffer_sentence = []
    for item in transcript_items:
        
        #filter on pronunciation, ignoring punctuation for the moment
        type_ = item['type']
        if type_ == 'pronunciation':
            start = float(item['start_time']) * 1000
            end = float(item['end_time']) * 1000
            content = item['alternatives'][0]['content']
            
            # splitting text across the different segments
            if start <= current_segment_end :
                buffer_sentence.append(content)
            else:
                if (len(buffer_sentence) > 0):
                    #appending "\r\n" at the end of each line - requirement from comprehend
                    sentence_list_per_segment.append(' '.join(buffer_sentence))
                buffer_sentence = []
                current_segment_end += segment_size_ms
                
    #flush the buffer at the end
    if (len(buffer_sentence) > 0):
        sentence_list_per_segment.append(' '.join(buffer_sentence))
    
    return sentence_list_per_segment

In [ ]:
#getting the transcript in the right format
video_transcript = prepare_transcribed_text_for_topic_modelling(transcript_items, segment_size_ms)

We're now writing the transcript in csv format in S3 to be consumed by Comprehend

In [ ]:
#writing the transcript in csv format in S3 to be consumed by Comprehend
def write_list_to_csv(local_file_path, rows, bucket, path):
    filename = local_file_path.split('/')[-1]
    #create file locally
    with open(local_file_path, 'w+') as f:
        write = csv.writer(f)
        for row in rows:
            write.writerow([row])
    #upload to S3
    boto3.resource('s3').Bucket(bucket).Object(os.path.join(path, filename)).upload_file(local_file_path)
    print(f"{filename} uploaded to s3://{bucket}/{path}/{filename}")

In [ ]:
transcript_filename = 'video_transcript.csv'
s3_comprehend_input_path = 'comprehend-input'

write_list_to_csv(os.path.join(tmp_local_folder, transcript_filename), 
                  video_transcript, 
                  bucket, 
                  s3_comprehend_input_path)

We're just checking the number of lines in the file we just created which should correspond to the duration of our video in minutes.

In [ ]:
num_lines = sum(1 for line in open(os.path.join(tmp_local_folder, transcript_filename)))
print(f'Number of lines in our file: {num_lines}')

# Comprehend - Topic detection
We're now ready to launch the first job.

You can use Amazon Comprehend to examine the content of a collection of documents to determine common themes. For example, you can give Amazon Comprehend a collection of news articles, and it will determine the subjects, such as sports, politics, or entertainment. The text in the documents doesn't need to be annotated.

Amazon Comprehend uses a Latent Dirichlet Allocation-based learning model to determine the topics in a set of documents. It examines each document to determine the context and meaning of a word. The set of words that frequently belong to the same context across the entire document set make up a topic.

https://docs.aws.amazon.com/comprehend/latest/dg/topic-modeling.html

In [ ]:
comprehend = boto3.client('comprehend')

s3_output_data_comprehend = os.path.join("s3://", bucket, 'comprehend-tm-output')
s3_input_data_comprehend = os.path.join("s3://", bucket, s3_comprehend_input_path)

#note that we're setting the number of topics to 15
response = comprehend.start_topics_detection_job(
    InputDataConfig={
        'S3Uri': s3_input_data_comprehend,
        'InputFormat': 'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': s3_output_data_comprehend,
    },
    DataAccessRoleArn=role_arn,
    JobName='comprehend_job_knowledge_graph_' + str(random.randint(0,100000)),
    NumberOfTopics=15
)


Monitoring the progress of the job

In [ ]:
while True:
    status = comprehend.describe_topics_detection_job(JobId=response['JobId'])
    if status['TopicsDetectionJobProperties']['JobStatus']  in ['COMPLETED', 'FAILED']:
        break
    print(".", end='')
    time.sleep(10)
print(comprehend.describe_topics_detection_job(JobId=response['JobId'])['TopicsDetectionJobProperties']['JobStatus'])

After Amazon Comprehend processes your document collection, it returns a compressed archive containing two files, topic-terms.csv and doc-topics.csv. 

The first output file, topic-terms.csv, is a list of topics in the collection. For each topic, the list includes by default the top terms by topic according to their weight. 

The second file, doc-topics.csv, lists the documents associated with a topic and the proportion of the document that is concerned with the topic. If you specified ONE_DOC_PER_FILE, the document is identified by the file name. If you specified ONE_DOC_PER_LINE (like in our case), the document is identified by the file name and the 0-indexed line number within the file. 

### Download and extract the comprehend topic detection output

In [ ]:
#function to extract a tar file
def extract(tar_file, path):
    opened_tar = tarfile.open(tar_file)
     
    if tarfile.is_tarfile(tar_file):
        opened_tar.extractall(path)
        return path
    else:
        print("The tar file you entered is not a tar file")

#download
def download_and_extract_comprehend_job_output(output_s3_uri, dl_path):
    s3_bucket = output_s3_uri.split('/')[2]
    s3_file_path = '/'.join(output_s3_uri.split('/', 3)[3:])
    local_file_path = os.path.join(dl_path, output_s3_uri.split('/')[-1])

    boto3.resource('s3').Bucket(s3_bucket).Object(s3_file_path).download_file(local_file_path)
    return extract(local_file_path, dl_path)

In [ ]:
topics_output_s3_uri = comprehend.describe_topics_detection_job(JobId=response['JobId'])['TopicsDetectionJobProperties']['OutputDataConfig']['S3Uri']

job_comprehend_output_folder = download_and_extract_comprehend_job_output(topics_output_s3_uri, tmp_local_folder)

Looking into the 2 output files and loading this into dataframes for later use.

In [ ]:
topics_file = 'doc-topics.csv'
topic_terms_file = 'topic-terms.csv'
comprehend_topics_df = pd.read_csv(os.path.join(tmp_local_folder, topics_file))
comprehend_terms_df = pd.read_csv(os.path.join(tmp_local_folder, topic_terms_file))

Displaying the 5 first documents and their topics

In [ ]:
comprehend_topics_df.head(5)

Displaying top 10 words for topic 1. This will give us an idea of what this topic is about. Remember that topic modelling is not outputing a specific label but instead an unlabeled topic or grouping of documents for which we have a list of prominent words and their weight/importance.

In [ ]:
comprehend_terms_df[comprehend_terms_df['topic'] == 1]

## Comprehend NER Named Entity Recognition

We're now looking at extracting Named entities from the video's transcript, still using Amazon Comprehend.

An entity is a textual reference to the unique name of a real-world object such as people, places, and commercial items, and to precise references to measures such as dates and quantities.

https://docs.aws.amazon.com/comprehend/latest/dg/how-entities.html

In [ ]:
response_NER = comprehend.start_entities_detection_job(
    InputDataConfig={
        'S3Uri': s3_input_data_comprehend,
        'InputFormat': 'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': s3_output_data_comprehend,
    },
    LanguageCode='en',
    DataAccessRoleArn=role_arn,
    JobName='comprehend_job_knowledge_graph_NER' + str(random.randint(0,100000)),
)

In [ ]:
while True:
    status_NER = comprehend.describe_entities_detection_job(JobId=response_NER['JobId'])
    if status_NER['EntitiesDetectionJobProperties']['JobStatus']  in ['COMPLETED', 'FAILED']:
        break
    print(".", end='')
    time.sleep(10)
print(comprehend.describe_entities_detection_job(JobId=response_NER['JobId'])['EntitiesDetectionJobProperties']['JobStatus'])

In [ ]:
#retrieving the outputs of our NER job
ner_output_s3_uri = comprehend.describe_entities_detection_job(JobId=response_NER['JobId'])['EntitiesDetectionJobProperties']['OutputDataConfig']['S3Uri']
job_comprehend_output_folder = download_and_extract_comprehend_job_output(ner_output_s3_uri, tmp_local_folder)

Let's look into the output of the NER job. As you can see we've got different types of entities: PERSON, DATE, QUANTITY, LOCATION, ORGANIZATION, OTHERS.

In [ ]:
ner_job_data = []
with jsonlines.open(os.path.join(tmp_local_folder, 'output')) as ner_json_reader:
    for obj in ner_json_reader:
        ner_job_data.append(obj['Entities'])

In [ ]:
ner_job_data[0]

Let's store that aside for it to be used in part 3 of the workshop

In [ ]:
%store segment_size_ms
%store comprehend_terms_df
%store comprehend_topics_df
%store ner_job_data